In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
plt.rcParams['figure.figsize'] = (20,6)

In [2]:
data = pd.read_csv('data/full_data.csv', delimiter = ';', parse_dates=True)

In [3]:
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [4]:
data.drop(columns='time', inplace=True)

In [5]:
data.head()

,timestamp,customer_no,location,weekday,weekday_num,customer_id
0,2019-09-02 07:03:00,1,dairy,Monday,0,0_1
1,2019-09-02 07:03:00,2,dairy,Monday,0,0_2
2,2019-09-02 07:04:00,3,dairy,Monday,0,0_3
3,2019-09-02 07:04:00,4,dairy,Monday,0,0_4
4,2019-09-02 07:04:00,5,spices,Monday,0,0_5


In [6]:
data.dtypes

timestamp      datetime64[ns]
customer_no             int64
location               object
weekday                object
weekday_num             int64
customer_id            object
dtype: object

In [7]:
# Number of unique customers
data['customer_id'].nunique()

7445

In [8]:
# Number of customers that are not "checked out"
#(data['customer_id'].nunique()) - (data[data['location'] == 'checkout'].value_counts().count())

In [9]:
# Drop these 28 customers

In [10]:
# Filter on valid customers
valid_customers = data[data['location'] == 'checkout'][['customer_id']]

In [11]:
# check if every customer_id is only included once
len(valid_customers)

7417

In [12]:
valid_customers['customer_id'].value_counts().sum()

7417

In [13]:
valid_cust = list(valid_customers['customer_id'])

In [14]:
#valid_cust

In [15]:
# Create new data set with valid customers only
data['mask'] = data['customer_id'].isin(valid_cust)

In [16]:
data.shape

(24877, 7)

In [17]:
valid_data = data[data['mask'] == True].copy()

In [18]:
valid_data.shape

(24798, 7)

In [19]:
valid_data['customer_id'].nunique()

7417

In [20]:
valid_data.head(3)

,timestamp,customer_no,location,weekday,weekday_num,customer_id,mask
0,2019-09-02 07:03:00,1,dairy,Monday,0,0_1,True
1,2019-09-02 07:03:00,2,dairy,Monday,0,0_2,True
2,2019-09-02 07:04:00,3,dairy,Monday,0,0_3,True


In [21]:
valid_data.dtypes

timestamp      datetime64[ns]
customer_no             int64
location               object
weekday                object
weekday_num             int64
customer_id            object
mask                     bool
dtype: object

In [22]:
valid_data['mask'].value_counts()

True    24798
Name: mask, dtype: int64

In [23]:
valid_data.drop(columns='mask', inplace=True)

In [24]:
######

In [25]:
# Add entrance (one minute before timestamp of first section in store)
df_entrance = valid_data.copy()

In [26]:
one_min = pd.Timedelta(minutes=1)

In [27]:
df_entrance['timestamp'] = df_entrance.groupby(['customer_id'])['timestamp'].transform('min')

In [28]:
df_entrance['timestamp'] = df_entrance['timestamp'] - one_min

In [29]:
df_entrance.drop_duplicates(subset=['customer_id'], inplace=True)

In [30]:
df_entrance['location']='entrance'

In [31]:
df_entrance['location'].value_counts()

entrance    7417
Name: location, dtype: int64

In [32]:
# Add dataframe with entrance data to "valid_data"

In [33]:
df_entrance.shape

(7417, 6)

In [34]:
valid_data.shape

(24798, 6)

In [35]:
df = pd.concat([valid_data, df_entrance])

In [36]:
df.shape

(32215, 6)

In [37]:
######

In [38]:
df.sort_values(['customer_id', 'timestamp'], inplace=True)

In [39]:
df.head(3)

,timestamp,customer_no,location,weekday,weekday_num,customer_id
0,2019-09-02 07:02:00,1,entrance,Monday,0,0_1
0,2019-09-02 07:03:00,1,dairy,Monday,0,0_1
8,2019-09-02 07:05:00,1,checkout,Monday,0,0_1


In [40]:
df.set_index('timestamp', inplace=True)

In [41]:
# To fill in rows for the minutes that are not in the files,
df = df.groupby('customer_id').resample('1min').fillna('ffill').copy()

In [42]:
df

customer_no  location weekday  weekday_num  \
customer_id timestamp                                                         
0_1         2019-09-02 07:02:00            1  entrance  Monday            0   
            2019-09-02 07:03:00            1     dairy  Monday            0   
            2019-09-02 07:04:00            1     dairy  Monday            0   
            2019-09-02 07:05:00            1  checkout  Monday            0   
0_10        2019-09-02 07:05:00           10  entrance  Monday            0   
...                                      ...       ...     ...          ...   
4_999       2019-09-06 17:23:00          999     fruit  Friday            4   
            2019-09-06 17:24:00          999    drinks  Friday            4   
            2019-09-06 17:25:00          999     fruit  Friday            4   
            2019-09-06 17:26:00          999     fruit  Friday            4   
            2019-09-06 17:27:00          999  checkout  Friday            4   

                                customer_id  
customer_id timestamp                        
0_1         2019-09-02 07:02:00         0_1  
            2019-09-02 07:03:00         0_1  
            2019-09-02 07:04:00         0_1  
            2019-09-02 07:05:00         0_1  
0_10        2019-09-02 07:05:00        0_10  
...                                     ...  
4_999       2019-09-06 17:23:00       4_999  
            2019-09-06 17:24:00       4_999  
            2019-09-06 17:25:00       4_999  
            2019-09-06 17:26:00       4_999  
            2019-09-06 17:27:00       4_999  

[61162 rows x 5 columns]

In [43]:
df.drop(columns='customer_id', inplace=True)

In [44]:
df.reset_index(inplace=True)

In [45]:
df.set_index('timestamp', inplace=True)

In [46]:
df.head(3)

,customer_id,customer_no,location,weekday,weekday_num
timestamp,,,,,
2019-09-02 07:02:00,0_1,1,entrance,Monday,0
2019-09-02 07:03:00,0_1,1,dairy,Monday,0
2019-09-02 07:04:00,0_1,1,dairy,Monday,0


In [47]:
# Add a column where customer was 'before' and 'after' (timestep is minutes)
df['before'] = df['location']
df['after'] = df['before'].shift(-1)
df.tail(20)

,customer_id,customer_no,location,weekday,weekday_num,before,after
timestamp,,,,,,,
2019-09-06 17:30:00,4_997,997,drinks,Friday,4,drinks,checkout
2019-09-06 17:31:00,4_997,997,checkout,Friday,4,checkout,entrance
2019-09-06 17:17:00,4_998,998,entrance,Friday,4,entrance,drinks
2019-09-06 17:18:00,4_998,998,drinks,Friday,4,drinks,fruit
2019-09-06 17:19:00,4_998,998,fruit,Friday,4,fruit,drinks
2019-09-06 17:20:00,4_998,998,drinks,Friday,4,drinks,drinks
2019-09-06 17:21:00,4_998,998,drinks,Friday,4,drinks,drinks
2019-09-06 17:22:00,4_998,998,drinks,Friday,4,drinks,checkout
2019-09-06 17:23:00,4_998,998,checkout,Friday,4,checkout,entrance


In [48]:
# Change 'after' to 'checkout' if customer had checked out before
df.loc[df['before'] == 'checkout', 'after'] = "checkout"

In [49]:
df.tail(20)

,customer_id,customer_no,location,weekday,weekday_num,before,after
timestamp,,,,,,,
2019-09-06 17:30:00,4_997,997,drinks,Friday,4,drinks,checkout
2019-09-06 17:31:00,4_997,997,checkout,Friday,4,checkout,checkout
2019-09-06 17:17:00,4_998,998,entrance,Friday,4,entrance,drinks
2019-09-06 17:18:00,4_998,998,drinks,Friday,4,drinks,fruit
2019-09-06 17:19:00,4_998,998,fruit,Friday,4,fruit,drinks
2019-09-06 17:20:00,4_998,998,drinks,Friday,4,drinks,drinks
2019-09-06 17:21:00,4_998,998,drinks,Friday,4,drinks,drinks
2019-09-06 17:22:00,4_998,998,drinks,Friday,4,drinks,checkout
2019-09-06 17:23:00,4_998,998,checkout,Friday,4,checkout,checkout


In [50]:
df.shape

(61162, 7)

In [51]:
df

,customer_id,customer_no,location,weekday,weekday_num,before,after
timestamp,,,,,,,
2019-09-02 07:02:00,0_1,1,entrance,Monday,0,entrance,dairy
2019-09-02 07:03:00,0_1,1,dairy,Monday,0,dairy,dairy
2019-09-02 07:04:00,0_1,1,dairy,Monday,0,dairy,checkout
2019-09-02 07:05:00,0_1,1,checkout,Monday,0,checkout,checkout
2019-09-02 07:05:00,0_10,10,entrance,Monday,0,entrance,fruit
...,...,...,...,...,...,...,...
2019-09-06 17:23:00,4_999,999,fruit,Friday,4,fruit,drinks
2019-09-06 17:24:00,4_999,999,drinks,Friday,4,drinks,fruit
2019-09-06 17:25:00,4_999,999,fruit,Friday,4,fruit,fruit


In [52]:
transition_matrix = pd.crosstab(df['after'], df['before'], normalize=1)

In [53]:
transition_matrix

before,checkout,dairy,drinks,entrance,fruit,spices
after,,,,,,
checkout,1.0,0.102968,0.216372,0.000000,0.201769,0.150464
dairy,0.0,0.737587,0.010943,0.286639,0.095475,0.193476
drinks,0.0,0.058339,0.598534,0.153566,0.054963,0.163096
fruit,0.0,0.049819,0.087437,0.378050,0.597094,0.090822
spices,0.0,0.051286,0.086714,0.181745,0.050699,0.402143


In [54]:
transition_matrix.sum(axis=0)

before
checkout    1.0
dairy       1.0
drinks      1.0
entrance    1.0
fruit       1.0
spices      1.0
dtype: float64

In [55]:
transition_matrix = transition_matrix.append(pd.Series(index=transition_matrix.columns, name='entrance')).fillna(0)


<ipython-input-55-104ef245ae61>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  transition_matrix = transition_matrix.append(pd.Series(index=transition_matrix.columns, name='entrance')).fillna(0)


In [63]:
transition_matrix.

before,checkout,dairy,drinks,entrance,fruit,spices
after,,,,,,
checkout,1.0,0.102968,0.216372,0.000000,0.201769,0.150464
dairy,0.0,0.737587,0.010943,0.286639,0.095475,0.193476
drinks,0.0,0.058339,0.598534,0.153566,0.054963,0.163096
fruit,0.0,0.049819,0.087437,0.378050,0.597094,0.090822
spices,0.0,0.051286,0.086714,0.181745,0.050699,0.402143
entrance,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [56]:
#########################################################################################
### Copied from Course code - to be adapted

In [57]:
# # transition prob matrix
# P = np.array([[0.4, 0.666],
#               [0.6, 0.333]])

# # initial state distribution (day 0)
# S = np.array([1.0, 0.0])   # <-- we start with a 100% sunny day

In [58]:
# S1 = np.dot(S, P.T)
# S1

In [59]:
# S2 = np.dot(S1, P.T)
# S2

In [60]:
# np.dot(np.dot(np.dot(np.dot(np.dot(S, P.T), P.T), P.T), P.T), P.T)

In [61]:
# result = []
# s = S.copy()
# for i in range(10):
#     result.append(s)
#     s = np.dot(s, P.T)

In [62]:
# pd.DataFrame(result).plot()